In [1]:
import os
import sys
root = '/home/mark/Documents/code/mdrp'
sys.path.append(root)

import numpy as np
from utils.nlp_mdrp import mdrp_prob
import cyipopt

from utils.convert import convert_instance
from utils.nlp_mdrp import mdrp_prob
from utils.helpers import *

In [2]:
def main(parameters):
    # Setup the directory and logger    
    # Load the instance

    isinstance_path = os.path.join(root+'/instances/',parameters['inst_path'])
    prob_setup = convert_instance(isinstance_path,parameters)
    # add additional parameters
    prob_setup['max_rho'] =  parameters['max_rho']
    prob_setup['max_cost'] = parameters['max_cost']
    prob_setup['demand'] =   parameters['demand']
    prob_setup['scale'] = parameters['scale']
    # SOLVE NLP
    mdrp = mdrp_prob(prob_setup)
    lb, ub, cl, cu = mdrp.get_bounds()
    x0 = mdrp.guess_init()

    nlp = cyipopt.Problem(
        n=len(x0),
        m=len(cl),
        problem_obj=mdrp,
        lb=lb,
        ub=ub,
        cl=cl,
        cu=cu
        )

    # nlp.addOption('derivative_test', 'second-order')
    nlp.add_option('mu_strategy', 'adaptive')
    nlp.add_option('tol', 1e-4)

    x, info = nlp.solve(x0)

    # Eval
    return x, mdrp

In [3]:
parameters = {}
parameters['speeds'] = [1,2,0.3]
parameters['N'] = [100,10,20]
parameters['u_ratios'] = [1,0.2,0.2]
parameters['max_rho'] = 0.9
parameters['max_cost'] = 10000
parameters['mode_costs'] = [10,20000,20000]
parameters['demand'] = 0.42
parameters['scale'] = False # if true then scales demand to be 0.99 of possible util

parameters['inst_path'] = '0o100t100s1p100'

N_values = ([100,10,20],[50,10,35],[20,20,35])
# costs are set arbitrarily large to not allow for cars
costs = ([10,20000,20000],[10,5,5],[10,5,5])

In [4]:
parameters['N'] = N_values[0]
parameters['mode_costs'] = costs[0]
sol_1, mdrp_1 = main(parameters)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:   765075
Number of nonzeros in inequality constraint Jacobian.:     6060
Number of nonzeros in Lagrangian Hessian.............:  1148370

Total number of variables............................:     1515
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1515
                     variables with only upper bounds:        0
Total number of equality constraints.................:      505
Total number of inequality c

In [5]:
parameters['N'] = N_values[1]
parameters['mode_costs'] = costs[1]
sol_2, mdrp_2 = main(parameters)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:   765075
Number of nonzeros in inequality constraint Jacobian.:     6060
Number of nonzeros in Lagrangian Hessian.............:  1148370

Total number of variables............................:     1515
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1515
                     variables with only upper bounds:        0
Total number of equality constraints.................:      505
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        4
        inequality constraints with only upper bounds:        0

Objective value at iteration #0 is - 0.324732
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.2473209e-01 3.47e-01 2.

In [6]:
parameters['N'] = N_values[2]
parameters['mode_costs'] = costs[2]
sol_3, mdrp_3 = main(parameters)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:   765075
Number of nonzeros in inequality constraint Jacobian.:     6060
Number of nonzeros in Lagrangian Hessian.............:  1148370

Total number of variables............................:     1515
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1515
                     variables with only upper bounds:        0
Total number of equality constraints.................:      505
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        4
        inequality constraints with only upper bounds:        0

Objective value at iteration #0 is - 0.301973
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.0197286e-01 5.69e-01 4.

In [7]:
def eval_sol_loc(x,mdrp,eval_setup):
    '''
    Inputs
        x          - solution as vector outputted by mdrp
        mdrp       - nlp_mdrp instance for the problem setting
                     contains useful methods
        mode_names - list of strings defining the name of the modes
    '''

    x = np.round(x,2)
    mat_x = mdrp._vec_to_mat(x)
    print('Problem Evlatuation')
    print('--------------'*4)
    demand = mdrp.demand*mdrp.n_orders
    print('Orders Demanded (order/hour): %.2f'%(demand))
    print('Maximum Cost ($/hour): %.2f'%mdrp.max_cost)
    print('--------------'*4)
    # Setup table
    print('Mode \t\t|', end="")
    for j in range(mdrp.n_modes):
        print('%s\t|'%eval_setup['mode_names'][j],end="")
    print('%s\t|'%'total',end="")
    print('\n----------------|',end="")
    for _ in range(mdrp.n_modes+1):
        print('-------|',end="")
    # Orders
    orders_j = mat_x.sum(axis=0)/demand # sum over orders per mode
    print('\nOrders (%)\t|',end="")
    for j in range(mdrp.n_modes):
        print('%.2f\t|'%(orders_j[j]),end="")
    # tot_util = mat_x.sum(axis=0).sum(axis=0)/(mdrp.mu*mdrp.N).sum()
    print('%.2f\t|'%orders_j.sum(),end="")

    # Cost
    print('\nOp. Cost ($/hr)\t|',end="")
    for j in range(mdrp.n_modes):
        cost_j = np.inner(mdrp.cost[:,j],mat_x[:,j])
        print('%.2f\t|'%(cost_j),end="")
    tot_cost = np.inner(mdrp._mat_to_vec(mdrp.cost),x)
    print('%.2f\t|'%tot_cost,end="")

    # Utilization
    util = mdrp._get_util(x)
    print('\nUtilization (%)\t|',end="")
    for j in range(mdrp.n_modes):
        print('%.2f\t|'%util[j],end="")
    tot_util = mat_x.sum(axis=0).sum(axis=0)/(mdrp.mu*mdrp.N).sum()
    print('%.2f\t|'%tot_util,end="")
    
    # Delivery Time
    lat = mdrp._get_latency(x)*60
    print('\nD Time(min.)\t|',end="")
    for j in range(mdrp.n_modes):
        lat_j = np.inner(lat[:,j],mat_x[:,j])/(orders_j[j]*demand)
        print('%.2f\t|'%lat_j,end="")
    tot_lat = mdrp.objective(x)*60
    print('%.2f\t|'%tot_lat,end="")

    # Prices
    prices = get_prices(x, mdrp, eval_setup)
    print('\nOrder Price\t|',end="")
    for j in range(mdrp.n_modes):
        price_j = np.inner(prices[:,j],mat_x[:,j])/(orders_j[j]*demand)
        print('%.2f\t|'%price_j,end="")
    tot_prices = np.inner(prices.flatten(),x)/(demand)
    print('%.2f\t|'%tot_prices,end="")
    # print("")
    # print(np.round(mat_x,2))
    # print(prices)

    # Delivery Distance
    mpm = 320 # meters per minute parameter
    distance = mdrp.t_time*mpm*60
    # print(pph)
    print('\nDistance\t|',end="")
    for j in range(mdrp.n_modes):
        dist_j = (eval_setup['speeds'][j]*np.inner(distance[:,j],mat_x[:,j]))/(orders_j[j]*demand*1000)
        # price_j = np.inner(distance[:,j],mat_x[:,j])/(orders_j[j]*demand)
        print('%.2f\t|'%dist_j,end="")
    tot_dist = np.inner(distance.flatten(),(mat_x*eval_setup['speeds']).flatten())/(demand*1000)
    print('%.2f\t|'%tot_dist,end="")

    tot_prices = np.inner(prices.flatten(),x)
    print('\nTotal Money from Orders: ', tot_prices)
    # Price per hour
    # pph = prices/(lat/60)
    # # print(pph)
    # print('\nPrice per hour\t|',end="")
    # for j in range(mdrp.n_modes):
    #     price_j = np.inner(pph[:,j],mat_x[:,j])/(orders_j[j]*demand)
    #     print('%.2f\t|'%price_j,end="")
    # tot_prices = np.inner(pph.flatten(),x)/(demand)
    # print('%.2f\t|'%tot_prices,end="")

    # print(mat_x[:,0])

In [20]:
eval_setup = {}
eval_setup['max_dph'] = 100
eval_setup['min_dph'] = 5
eval_setup['min_price'] = 10
eval_setup['mode_names'] = ['cars','drones','droids']
eval_setup['speeds'] = [1,2,0.3]

print('Cars only')

eval_sol_loc(sol_1,mdrp_1,eval_setup)

print('\n')
print('Now with Drones and Robots')
eval_setup['min_price'] = 5.42
eval_sol_loc(sol_2,mdrp_2,eval_setup)

print('\n')
print('Now with More Drones and Robots')
eval_setup['min_price'] = 3.95
eval_sol_loc(sol_3,mdrp_3,eval_setup)

Cars only
Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 212.10
Maximum Cost ($/hour): 10000.00
--------------------------------------------------------
Mode 		|cars	|drones	|droids	|total	|
----------------|-------|-------|-------|-------|
Orders (%)	|1.00	|0.00	|0.00	|1.00	|
Op. Cost ($/hr)	|2121.00	|0.00	|0.00	|2121.00	|
Utilization (%)	|0.88	|0.00	|0.00	|0.64	|
D Time(min.)	|21.50	|nan	|nan	|21.50	|
Order Price	|10.96	|nan	|nan	|10.96	|
Distance	|2.36	|nan	|nan	|2.36	|
Total Money from Orders:  2324.6926434282595


Now with Drones and Robots
Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 212.10
Maximum Cost ($/hour): 10000.00
--------------------------------------------------------
Mode 		|cars	|drones	|droids	|total	|
----------------|-------|-------|-------|-------|
Orders (%)	|0.50	|0.24	|0.26	|1.00	|
Op. Cost ($/hr)	|1061.90	|255.85	|276.10	|1593.85	|
Utilizat

/tmp/ipykernel_10002/844613600.py:54: RuntimeWarning: invalid value encountered in scalar divide
  lat_j = np.inner(lat[:,j],mat_x[:,j])/(orders_j[j]*demand)
/tmp/ipykernel_10002/844613600.py:63: RuntimeWarning: invalid value encountered in scalar divide
  price_j = np.inner(prices[:,j],mat_x[:,j])/(orders_j[j]*demand)
/tmp/ipykernel_10002/844613600.py:77: RuntimeWarning: invalid value encountered in scalar divide
  dist_j = (eval_setup['speeds'][j]*np.inner(distance[:,j],mat_x[:,j]))/(orders_j[j]*demand*1000)


In [11]:
(1600-1267)/212

1.570754716981132